<a href="https://colab.research.google.com/github/annies99/c/blob/master/Lightweight_Model_for_Predicting_Sound_%5BReplicable%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Install necessary libraries
!pip install torch torchaudio torchvision

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
import torchaudio
import numpy as np
import os
import matplotlib.pyplot as plt

# Set device (GPU if available, otherwise CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define transformations for the image data
data_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the dataset (replace '/path/to/dataset' with the actual path)
data_dir = '/content/drive/MyDrive/ImagesDataset'
image_dataset = datasets.ImageFolder(os.path.join(data_dir), transform=data_transforms)

# Create DataLoader
batch_size = 32
dataloader = torch.utils.data.DataLoader(image_dataset, batch_size=batch_size, shuffle=True)

# Load pre-trained ResNet50 model
resnet50 = models.resnet50(pretrained=True)
num_features = resnet50.fc.in_features

# Remove the final fully connected layer
resnet50 = nn.Sequential(*list(resnet50.children())[:-1])
resnet50 = resnet50.to(device)

# Define a simple linear regression model to predict sounds from image features
class SoundPredictionModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(SoundPredictionModel, self).__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        return self.linear(x)

# Initialize the sound prediction model
sound_model = SoundPredictionModel(num_features, 1)
sound_model = sound_model.to(device)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(sound_model.parameters(), lr=0.001)

# Train the model
num_epochs = 10

for epoch in range(num_epochs):
    running_loss = 0.0

    for inputs, _ in dataloader:
        inputs = inputs.to(device)

        # Extract features from the images using ResNet50
        with torch.no_grad():
            features = resnet50(inputs)

        # Flatten the features
        features = features.view(features.size(0), -1)

        # Forward pass
        outputs = sound_model(features)

        # Generate random target sound (replace with actual target sound)
        targets = torch.randn(outputs.size(0), 1).to(device)

        # Compute loss
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(image_dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

print('Training complete!')

In [ ]:
import cv2
import torch
import numpy as np
from torchvision import transforms
from model import SoundPredictionModel  # Assuming the trained model is saved as model.pth

# Load the trained sound prediction model
sound_model = SoundPredictionModel(input_size, output_size)
sound_model.load_state_dict(torch.load('sound_prediction_model.pth'))
sound_model.eval()

# Define a transform to preprocess each frame of the video
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the video file
video_path = 'path_to_your_video.mp4'
cap = cv2.VideoCapture(video_path)

# Create an empty list to store the predicted sounds
predicted_sounds = []

# Process each frame of the video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess the frame
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = transform(frame)

    # Add a batch dimension and move the frame to the GPU if available
    frame = frame.unsqueeze(0)
    frame = frame.to(device)

    # Extract features from the frame using the pre-trained ResNet50 model
    with torch.no_grad():
        features = resnet50(frame)

    # Flatten the features
    features = features.view(features.size(0), -1)

    # Predict the sound corresponding to the frame
    sound_prediction = sound_model(features)

    # Convert the prediction to numpy array and append to the list
    predicted_sound = sound_prediction.cpu().numpy().flatten()
    predicted_sounds.append(predicted_sound)

# Combine the predicted sounds into a single audio file (you may need additional libraries for this)
# Save or play the audio file